In [1]:
%matplotlib notebook
from spatialmath import *
from spatialmath.base import *
import matplotlib.pyplot as plt
from math import pi, degrees, radians
import numpy as np
import sympy as sp

# extendemos la clase SE3 para poder manejar variables en forma simbolica y numérica
# USO: objeto.symbolReplace([(simbolo1, valor_simbolo1),(simboloN, valor_simboloN)])
class SE3(SE3):
    def symbolReplace(self,symbol_values):
        aux = np.eye(self.A.shape[0])
        for i in range(self.A.shape[0]):
            for j in range(self.A.shape[1]):
                try:
                    aux[i,j] = self.A[i,j].subs(symbol_values).evalf()
                except:
                    pass
        return aux

In [2]:
# Definimos simbolos a utilizar
q1, q2, q3 = sp.symbols('Q1 Q2 Q3')

In [3]:
A01 = SE3.Rz(q1) * SE3.Tz(200) * SE3.Tx(100) * SE3.Rx(pi/2)
A01

  1.0*cos(Q1)  -6.12323399573677e-17*sin(Q1) 1.0*sin(Q1)  100.0*cos(Q1)  
  1.0*sin(Q1)  6.12323399573677e-17*cos(Q1) -1.0*cos(Q1) 100.0*sin(Q1)  
   0         1         0         200       
   0         0         0         1         


In [4]:
A12 = SE3.Rz(q2) * SE3.Tx(100)
A12

  1.0*cos(Q2)  -1.0*sin(Q2) 0            100.0*cos(Q2)  
  1.0*sin(Q2)  1.0*cos(Q2)  0            100.0*sin(Q2)  
   0         0         1         0         
   0         0         0         1         


In [5]:
A23 = SE3.Rz(q3) * SE3.Tz(100) * SE3.Rx(pi)
A23

  1.0*cos(Q3)  1.0*sin(Q3)  1.22464679914735e-16*sin(Q3) 0             
  1.0*sin(Q3)  -1.0*cos(Q3) -1.22464679914735e-16*cos(Q3) 0             
   0         0        -1         100       
   0         0         0         1         


In [44]:
# Lo que nos interesa de T es lo remarcado en azul, es decir, los componentes de P
T = A01 * A12 * A23
T

  1.0*(-6.12323399573677e-17*sin(Q1)*sin(Q2) + 1.0*cos(Q1)*cos(Q2))*cos(Q3) + 1.0*(-6.12323399573677e-17*sin(Q1)*cos(Q2) - 1.0*sin(Q2)*cos(Q1))*sin(Q3) 1.0*(-6.12323399573677e-17*sin(Q1)*sin(Q2) + 1.0*cos(Q1)*cos(Q2))*sin(Q3) - 1.0*(-6.12323399573677e-17*sin(Q1)*cos(Q2) - 1.0*sin(Q2)*cos(Q1))*cos(Q3) + 1.22464679914735e-16*sin(Q1) 1.22464679914735e-16*(-6.12323399573677e-17*sin(Q1)*sin(Q2) + 1.0*cos(Q1)*cos(Q2))*sin(Q3) - 1.22464679914735e-16*(-6.12323399573677e-17*sin(Q1)*cos(Q2) - 1.0*sin(Q2)*cos(Q1))*cos(Q3) - 1.0*sin(Q1) -6.12323399573677e-15*sin(Q1)*sin(Q2) + 100.0*sin(Q1) + 100.0*cos(Q1)*cos(Q2) + 100.0*cos(Q1)  
  1.0*(-1.0*sin(Q1)*sin(Q2) + 6.12323399573677e-17*cos(Q1)*cos(Q2))*sin(Q3) + 1.0*(1.0*sin(Q1)*cos(Q2) + 6.12323399573677e-17*sin(Q2)*cos(Q1))*cos(Q3) -1.0*(-1.0*sin(Q1)*sin(Q2) + 6.12323399573677e-17*cos(Q1)*cos(Q2))*cos(Q3) + 1.0*(1.0*sin(Q1)*cos(Q2) + 6.12323399573677e-17*sin(Q2)*cos(Q1))*sin(Q3) - 1.22464679914735e-16*cos(Q1) -1.22464679914735e-16*(-1.0*sin(Q1)*sin(Q

In [45]:
# Asignamos P como filas 0 a 3 (sin incluir) y columna 3 del tensor T
P = T.A[:3,3]
# Aplicamos simplificacion algebraica y numerica
P = [i.simplify().nsimplify(tolerance = 1e-10).simplify() for i in P]

In [46]:
# Desempaquetamos P
px,py,pz = P

In [47]:
# Coordenada X del efector segun Q
px

100*sqrt(2)*sin(Q1 + pi/4) + 100*cos(Q1)*cos(Q2)

In [48]:
# Coordenada Y del efector segun Q
py

100*sin(Q1)*cos(Q2) - 100*sqrt(2)*cos(Q1 + pi/4)

In [49]:
# Coordenada Z del efector segun Q
pz

100*sin(Q2) + 200

In [50]:
np.set_printoptions(suppress=True)

In [51]:
px1_eval = px.subs([('Q1',0),('Q2',pi/2),('Q3',0)]).round()
px1_eval

100

In [52]:
py1_eval = py.subs([('Q1',0),('Q2',pi/2),('Q3',0)]).round()
py1_eval

-100

In [53]:
pz1_eval = pz.subs([('Q1',0),('Q2',pi/2),('Q3',0)]).round()
pz1_eval

300

In [54]:
T.symbolReplace([('Q1',0),('Q2',pi/2),('Q3',0)]).round(3)

array([[   0.,    1.,    0.,  100.],
       [   0.,   -0.,    1., -100.],
       [   1.,   -0.,   -0.,  300.],
       [   0.,    0.,    0.,    1.]])